In [ ]:
import numpy as np
import pandas as pd

In [ ]:
class Interest:
    name = 'unnamed'
    tags = []
    sub_interests = []

    def __init__(self, name, tags, sub_interests):
        self.name = name
        self.tags = tags
        self.sub_interests = sub_interests

#Create some interests

interests_list = [
    #Interest(name, tags, sub_interests)
    Interest('space', ['science', 'space'], ['stars', 'planets']),
    Interest('biology', ['science', 'biology'], ['humans', 'animals', 'plants', 'insects']),
    Interest('braingames', ['math', 'braingames'], ['maths', 'physics', 'chemistry', 'chess']),
    Interest('art', ['art', 'creativity'], ['drawing', 'handcrafts', 'sculpture', 'music']),
    Interest('music', ['creativity', 'art'], ['playingmusic', 'musictheory'])
]

In [ ]:
credits = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_credits.csv')
movies_df = pd.read_csv('../input/tmdb-movie-metadata/tmdb_5000_movies.csv')

In [ ]:
credits.head()

In [ ]:
movies_df.head()

In [ ]:
credits_column_renamed = credits.rename(index=str, columns={"movie_id" : 'id'})
movies_df_merge = movies_df.merge(credits_column_renamed, on = 'id')
movies_df_merge.head()

In [ ]:
movies_cleaned_df = movies_df_merge.drop(columns=['homepage', 'title_x', 'title_y', 'status', 'production_countries'])
movies_cleaned_df.head()

## movies_cleaned_df.info()

In [ ]:
movies_cleaned_df.head(2)['overview']

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfv = TfidfVectorizer(min_df=3, max_features=None, strip_accents='unicode',
                     analyzer='word', token_pattern=r'\w{1,}', ngram_range=(1, 3),
                     stop_words='english')

movies_cleaned_df['overview'] = movies_cleaned_df['overview'].fillna('')

In [ ]:
tfv_matrix = tfv.fit_transform(movies_cleaned_df['overview'])

In [ ]:
tfv_matrix

In [ ]:
from sklearn.metrics.pairwise import sigmoid_kernel

sig = sigmoid_kernel(tfv_matrix, tfv_matrix)

In [ ]:
sig[0]

In [ ]:
indices = pd.Series(movies_cleaned_df.index, index=movies_cleaned_df['original_title']).drop_duplicates()

In [ ]:
indices

In [ ]:
def give_rec(title, sig=sig):
    idx = indices[title]
    sig_scores = list(enumerate(sig[idx]))
    
    sig_scores = sorted(sig_scores, key=lambda x:x[1], reverse=True)
    
    sig_scores = sig_scores[1:11]
    
    movie_indices = [i[0] for i in sig_scores]
    
    return movies_cleaned_df['original_title'].iloc[movie_indices]

In [ ]:
give_rec('Spy Kids')